### **Intall Dependencies**



In [ ]:
! pip install -r "/content/drive/MyDrive/mini-project-01/requirements.txt"

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.7/603.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.7/496.7 kB 47.2 MB/s eta 0:00:00
   ━━

### **Import Reuqired Libraries**

In [ ]:
import sys
import torch
from datasets import load_dataset,Dataset
from pathlib import Path
import yaml
import pandas as pd
from transformers import  AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os
import numpy as np
from peft import prepare_model_for_kbit_training,LoraConfig
from huggingface_hub import upload_folder
from google.colab import userdata
from trl import SFTTrainer,SFTConfig
from peft import get_peft_model,PeftModel
import time
import warnings

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
##ignore warnings
warnings.filterwarnings("ignore")

### **Enviroment Check**

In [ ]:
print("="*60)
print(f"Python Version:{sys.version}")
print(f"Pytorch Version:{torch.__version__}")
print(f"CUDA is Available:{torch.cuda.is_available()}")
print("="*60)

print("="*60)
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"Device capability: {torch.cuda.get_device_capability(0)}")
    print(f"Total VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("CUDA is not available!!!!!!!!!!!!!!!")
print("="*60)

Python Version:3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pytorch Version:2.9.0+cu126
CUDA is Available:True
CUDA version: 12.6
Device name: NVIDIA A100-SXM4-80GB
Device capability: (8, 0)
Total VRAM: 79.32 GB


In [ ]:
##current working directory
PROJECT_ROOT=Path.cwd()/'drive/MyDrive/mini-project-01'
os.chdir(PROJECT_ROOT)

print(f"Current working directory:{PROJECT_ROOT}")

Current working directory:/content/drive/MyDrive/mini-project-01


### **Load the yaml file**

In [ ]:
## Load the yaml file
def load_yaml(file_path):
    with open(file_path,'r') as file:
        config=yaml.safe_load(file)

    return config


##Load the config file
config=load_yaml(PROJECT_ROOT/"config.yaml")

### **Load the Dataset**

In [ ]:
dataset=load_dataset(
    "json",
    data_files={"train":"/content/drive/MyDrive/mini-project-01/data/final/train.jsonl"}
)

train_dataset=dataset["train"]

## save the dataset as pandas dataframe
train_dataset=pd.DataFrame(train_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print("="*60)
print(f"Display the First {len(train_dataset[:10])} rows in the train dataset:")
print("="*60)
train_dataset[:10]

Display the First 10 rows in the train dataset:


,chunk_id,question_id,question,anwser
0,49,4,What percentage of revenue was allocated to ge...,The percentage of revenue allocated to general...
1,104,6,What potential impact could litigation and arb...,Litigation and arbitration could have a materi...
2,21,4,What type of offerings does the company provid...,The company provides Mobility products and Del...
3,42,4,What could result from the issuance of additio...,The issuance of additional shares of common st...
4,90,10,Describe the potential financial implications ...,The potential financial implications of recogn...
5,39,6,What might happen to the company's dynamic pri...,The dynamic pricing strategy may be challenged...
6,65,8,Given the decrease in total equity from 2022 t...,Given the decrease in total equity from 2022 t...
7,63,3,What specific aspects of the consolidated fina...,The audits assess the risks of material missta...
8,40,2,What types of emergencies and conditions are m...,The types of emergencies and conditions mentio...
9,3,8,What are the potential risks associated with c...,The potential risks associated with compliance...


### **Huggingface Login**

In [ ]:
print("="*60)
print("Huggingface Login............")
hf_token=userdata.get('HF_TOKEN')
!hf auth login --token $hf_token

Huggingface Login............
A new version of huggingface_hub (1.4.1) is available! You are using version 1.3.7.
To update, run: pip install -U huggingface_hub

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Llama-model-4bit` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Llama-model-4bit`


### **Check the Token Length Diagnostics**

In [ ]:
print(f"Loading Tokenizer:{config["base_model"]}")
tokenizer=AutoTokenizer.from_pretrained(config["base_model"],trust_remote_code=True)

Loading Tokenizer:meta-llama/Meta-Llama-3-8B-Instruct


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token=tokenizer.eos_token

In [ ]:
tokens_lengths=[]
for _,example in train_dataset.iterrows():
    text=f"{example["question"]} {example["anwser"]}"
    tokens=tokenizer(text,add_special_tokens=True)
    tokens_lengths.append(len(tokens["input_ids"]))

tokens_lengths=np.array(tokens_lengths)

print("="*60)
print("Token Length Diagnostics")
print("="*60)

print(f"Sample Size:{len(train_dataset)}")
print(f"Average Token Lenght:{tokens_lengths.mean():.1f}")
print(f"Median Token Length:{np.median(tokens_lengths)}")
print(f"Minimum Token Length:{np.min(tokens_lengths)}")
print(f"Maximum Token Length:{np.max(tokens_lengths)}")
print(f"95th percentile of the Token Length:{np.percentile(tokens_lengths,95):.1f}")
print(f"99th percentile of the Token Length:{np.percentile(tokens_lengths,99)}")


Token Length Diagnostics
Sample Size:904
Average Token Lenght:43.7
Median Token Length:42.0
Minimum Token Length:14
Maximum Token Length:89
95th percentile of the Token Length:65.8
99th percentile of the Token Length:78.0


### **Build the prompt format for SFTTrainer**

In [ ]:
def prompt_format(user_text,system_prompt="You are a helpful financial analyst.",assistant_text=None,generation_prompt=False):
  ##----Format of CHAT ML------------
  messages=[
      {"role":"system","content":system_prompt},
      {"role":"user","content":user_text}
  ]
  if assistant_text is not None:
    messages.append({"role":"assistant","content":assistant_text})

  return tokenizer.apply_chat_template(messages,tokenize=False,add_generation_prompt=generation_prompt)


def build_sftt_prompt(row):
    """
     Build the SFTTrainer promtp format using dataset row.
     args:row
     type:str
     return:prompt
     type:str
    """

    user_text=row["question"]
    assistant_text=row["anwser"]

    prompt=prompt_format(
        user_text=user_text,
        system_prompt="You are a helpful financial analyst.",
        assistant_text=assistant_text
    )

    return {"text":prompt}


train_dataset_prompt=train_dataset.apply(build_sftt_prompt,axis=1)
##val_dataset_prompt=test_dataset.apply(build_sftt_prompt,axis=1)

print("="*60)
print(f"Prompts Build for SFT Trainer")
print("="*60)
print(f"Sample Formatted of Prompt:")
print("="*60)
print(train_dataset_prompt[0]["text"][:500]+".....")


Prompts Build for SFT Trainer
Sample Formatted of Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful financial analyst.<|eot_id|><|start_header_id|>user<|end_header_id|>

What percentage of revenue was allocated to general and administrative expenses in 2024?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The percentage of revenue allocated to general and administrative expenses in 2024 was 8%.<|eot_id|>.....


### **Load the model and apply 4-bit Quantization**

In [ ]:
## 4-bit Quantization configuration
bnb_config=BitsAndBytesConfig(
    load_in_4bit=config["load_in_4bit"],
    bnb_4bit_compute_dtype=config["bnb_4bit_compute_dtype"],
    bnb_4bit_quant_type=config["bnb_4bit_quant_type"],
    bnb_4bit_use_double_quant=config["bnb_4bit_use_double_quant"]
)

## Load the 4-bit quantized model
print(f"Loading base model:{config["base_model"]}.....")

model = AutoModelForCausalLM.from_pretrained(
    config["base_model"],
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype="auto"
)
print("Loaded 4-bit quantized model")

Loading base model:meta-llama/Meta-Llama-3-8B-Instruct.....


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Loaded 4-bit quantized model


In [ ]:
# Convert pandas Series to Hugging Face Dataset
train_dataset_hf = Dataset.from_list(train_dataset_prompt.tolist())
#val_dataset_hf = Dataset.from_list(train_dataset_prompt.tolist())

### **Prepare Model for LORA Training**

In [ ]:
print("Preparing a model for k-bit Training.....")
model=prepare_model_for_kbit_training(model)

## LORA configurations
lora_config=LoraConfig(
    r=config["lora_r"],
    lora_alpha=config["lora_alpha"],
    lora_dropout=config["lora_dropout"],
    target_modules=config["lora_target_modules"],
    bias="none",
    task_type="CAUSAL_LM"
)

## Apply LOARA
print("="*60)
print("Applying LORA adapters...")
model=get_peft_model(model,lora_config)
model.print_trainable_parameters()
print("="*60)

##Applying Training Arguments
training_args=SFTConfig(
    output_dir=config["output_dir"],
    per_device_train_batch_size=config["per_device_train_batch_size"],
    gradient_accumulation_steps=config["gradient_accumulation_steps"],
    learning_rate=float(config["learning_rate"]),
    num_train_epochs=config["num_train_epoch"],
    max_steps=config["max_steps"],
    logging_steps=config["logging_step"],
    save_steps=config["save_steps"],
    fp16=False,
    bf16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    lr_scheduler_type="cosine",
    packing=False,
    dataset_text_field="text",
)

##Create Trainer
trainer=SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_hf
)

Preparing a model for k-bit Training.....
Applying LORA adapters...
trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196


Adding EOS to train dataset:   0%|          | 0/904 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/904 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/904 [00:00<?, ? examples/s]

In [ ]:
print("="*60)
print("TRAINING CONFIGURATION")
print("="*60)
print(f"Effective batch size:{config["per_device_train_batch_size"]*config["gradient_accumulation_steps"]}")
print(f"Training steps:{config["max_steps"]}")
print(f"Total samples:{config["max_steps"]*config["per_device_train_batch_size"]*config["gradient_accumulation_steps"]}")
print(f"Optimizer: paged_adamw_8bit")
print(f"Learning rate:{config["learning_rate"]}")
print(f"LoRA rank:{config["lora_r"]}")
print("="*60)

# Train
print("\n🚀 Starting training...\n")
train_result = trainer.train()

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)
print(f"Total training time: {train_result.metrics.get('train_runtime', 0):.2f}s")
print(f"Samples per second: {train_result.metrics.get('train_samples_per_second', 0):.2f}")
print(f"Steps per second: {train_result.metrics.get('train_steps_per_second', 0):.4f}")

TRAINING CONFIGURATION
Effective batch size:8
Training steps:150
Total samples:1200
Optimizer: paged_adamw_8bit
Learning rate:2e-4
LoRA rank:16

🚀 Starting training...



Step,Training Loss
10,2.119056
20,1.174293
30,1.092363
40,1.029296
50,0.952174
60,0.877987
70,0.869163
80,0.895512
90,0.906642
100,0.930125



TRAINING COMPLETE
Total training time: 521.84s
Samples per second: 2.30
Steps per second: 0.2870


### **Save Adapters and Tokenizer**

In [ ]:
## Save Adapter and tokenizer
print("="*60)
print(f"Saving Adapters to {config["output_dir"]}....")
trainer.model.save_pretrained(config["output_dir"])
tokenizer.save_pretrained(config["output_dir"])
print("Saved Adapter and Toknizer")
print("="*60)


## Create a model card and saved
model_card=f"""
---
license: Llama 3 Community License
tags:
- finance
- llm
- qlora
- llama3
- instruction-tuning
- annual-report
---

# Uber 2024 Annual Report – LLaMA-3 QLoRA Adapter

This repository contains a **LoRA (Low-Rank Adaptation) adapter** fine-tuned on
**instruction–answer pairs derived from Uber’s 2024 Annual Report**.
The adapter specializes a general-purpose LLaMA-3-8B model to better understand
Uber-specific strategy, financial performance, regulatory risks, and corporate tone.

---

## Base Model

**{config['base_model']}**
This adapter must be loaded on top of the base model.

---

## Dataset

The training dataset consists of **synthetic instruction–response pairs**
generated from cleaned and chunked sections of the **Uber 2024 Annual Report**.

### Dataset Characteristics
- Source: Uber 2024 Annual Report (PDF)
- Data type: Single-turn instruction–answer pairs
- Question types:
  - Factual (financial figures, entities)
  - Strategic / analytical
  - Stylistic / summarization
- Train/Test split: Created at chunk level to prevent leakage

The dataset was generated using a two-stage LLM pipeline and manually validated
for factual faithfulness and hallucination risk.
---
## Training Method

- Fine-tuning method: **QLoRA**
- Trainer: `trl.SFTTrainer`
- Quantization: **4-bit NF4 with double quantization**
- Objective: Supervised instruction fine-tuning

---

## Training Hyperparameters

- LoRA rank (r): {config['lora_r']}
- LoRA alpha: {config['lora_alpha']}
- LoRA dropout: {config['lora_dropout']}
- Target modules: q_proj, k_proj, v_proj, o_proj
- Learning rate: {config['learning_rate']}
- Max training steps: {config['max_steps']} (≥100)
- Effective batch size: {config["per_device_train_batch_size"]*config["gradient_accumulation_steps"]}
- Max sequence length: {config['max_token_length']}

---

## Usage

```python
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer,SFTConfig
from peft import get_peft_model
from peft import prepare_model_for_kbit_training,LoraConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "{config['base_model']}",
    quantization_config=bnb_config,
    device_map="auto"
)

model=prepare_model_for_kbit_training(model)
model=get_peft_model(model,lora_config)
tokenizer = AutoTokenizer.from_pretrained("{config['base_model']}")

prompt = \"\"\"### Instruction:
What were Uber’s key strategic priorities in 2024?

### Response:
\"\"\"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=384)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Intended Use:
-This model is intended for:
-**Educational purposes**
-Research on instruction tuning
-Analysis of corporate disclosures and financial narratives

### Limitations & Disclaimer:
-This model does not provide financial advice
-Outputs may be incomplete or incorrect
-Trained on a limited, document-specific dataset
-Should not be used for investment or legal decision-making

"""
print("="*60)
print("Saving model card.....")
with open(os.path.join(config["output_dir"],"README.md"),"w") as f:
  f.write(model_card)

print("Model card saved")
print("="*60)


##Model push to the huggging face hub
config["push_to_hub"]=True
if config["push_to_hub"]:
  ##create a repo name
   repo_id=f"{config["hf_username"]}/{config["hub_model_name"]}"

   ##pushing to the hugging face
   print(f"Pushing to the hugging face Hub:{repo_id}")
   trainer.model.push_to_hub(repo_id)
   tokenizer.push_to_hub(repo_id)

   print(f"✅ Pushed to Hub: https://huggingface.co/{repo_id}")

Saving Adapters to output/adapters....
Saved Adapter and Toknizer
Saving model card.....
Model card saved
Pushing to the hugging face Hub:Hemalmewan/uber_finetune_model


README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   1%|          |  607kB / 83.9MB            

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpxk2ynfx8/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

No files have been modified since last commit. Skipping to prevent empty commit.


✅ Pushed to Hub: https://huggingface.co/Hemalmewan/uber_finetune_model


### **Inference Pipeline(Query Intern)**

In [ ]:
##del model
##del trainer
torch.cuda.empty_cache()

print("Loading base model...")
base_model_inference = AutoModelForCausalLM.from_pretrained(
    config["base_model"],
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
    )

print("Loading finetuned adapter...")
finetuned_model = PeftModel.from_pretrained(base_model_inference, f"{config['hf_username']}/{config['hub_model_name']}")
print("Finetuned model ready")

def query_intern(user_input,system_prompt="You are a helpful financial analyst."):

    finetuned_model.model.eval()

    # Apply chat template to convert messages into a single formatted string
    formatted_prompt =prompt_format(user_text=user_input,
                                      system_prompt=system_prompt,
                                      generation_prompt=True)

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(finetuned_model.device)

    terminators = [
          tokenizer.eos_token_id,
          tokenizer.convert_tokens_to_ids("<|eot_id|>")
      ]

    with torch.no_grad():
        outputs = finetuned_model.generate(
            **inputs,
            max_new_tokens=config["max_tokens"],
            do_sample=config["do_sample"],
            temperature=config["temperature"],
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=terminators,

          )

    new_tokens = outputs[0][len(inputs.input_ids[0]):]

    gen_text = tokenizer.decode(new_tokens, skip_special_tokens=True)

    return gen_text

Loading base model...


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Loading finetuned adapter...


adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

Finetuned model ready


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

### **Evaluation using ROUGE-L and LLM-as a judge**

In [ ]:
dataset=load_dataset(
    "json",
    data_files={"test":"/content/drive/MyDrive/mini-project-01/data/final/test.jsonl"}
)

## save the dataset as pandas dataframe
test_dataset=pd.DataFrame(dataset["test"])

Generating test split: 0 examples [00:00, ? examples/s]

### **Check the inference time**

In [ ]:
def run_baseline_model(question):
    start = time.time()
    answer = query_intern(question)   # no retrieval
    latency = (time.time() - start) * 1000
    return answer, latency

### **Evaluating using ROUGE-L**

In [ ]:
from rouge_score import rouge_scorer

scorer=rouge_scorer.RougeScorer(['rougeL'],use_stemmer=True)

def rouge_l(prediction, reference):
    return scorer.score(reference, prediction)["rougeL"].fmeasure

In [ ]:
i=0
for row in test_dataset.iterrows():
  answer,latency=run_baseline_model(row[1]["question"])

  print(f"Customer Question:{row[1]["question"]}")
  print(f"Assistant Answer:{answer}")
  print(f"Latency:{latency:.2f} ms")
  print(f"Reference Answer:{row[1]['anwser']}")
  print(f"Rouge-L Score:{rouge_l(answer,row[1]['anwser']):.4f}")
  print("="*60)

  i+=1
  if i==10:
    break


Customer Question:How many trading days must the price of the common stock exceed for at least 20 days during a 30-day period for a specific conversion condition?
Assistant Answer:The price of the common stock must exceed for at least 20 trading days during a 30-day period.
Latency:4287.53 ms
Reference Answer:The price of the common stock must exceed for at least 20 trading days during a 30-day period for a specific conversion condition.
Rouge-L Score:0.8837
Customer Question:Summarize the implications of failing to comply with healthcare-related regulations as outlined in the report.
Assistant Answer:Failing to comply with healthcare-related regulations could lead to fines, penalties, and negative publicity, which could harm the company's reputation and business.
Latency:3947.82 ms
Reference Answer:Failing to comply with healthcare-related regulations may result in government authorities assessing significant fines.
Rouge-L Score:0.4103
Customer Question:What was the balance of intang

### **Evaluating using LLM-as a judge**

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI

## Openai API key
load_dotenv()
openai_api_key=os.getenv("OPENAI_API_KEY")

if not openai_api_key:
  print("Warning:no API key is found in .env file")
  print("Looking for a OpenAI API key....")
  gemini_api_key=input("Enter your OpenAI API key:")

  use_llm_as_judge=False
  judge_model=False

else:
  ## Configure Openai with new API key
  client_ai=OpenAI(api_key=openai_api_key)
  judge_model=client_ai
  model_name="gpt-4o-mini"
  use_llm_as_judge=True

  print(f"OpenAI API key configure with:{model_name}")
  print(f"Using OpenAI package")

OpenAI API key configure with:gpt-4o-mini
Using OpenAI package


In [ ]:
## define the function for llm-as a judge
def llm_as_judge(question,prediction,reference):
  """
   Use a OpenAI model(GPT-4o-mini) as judge to check the quality of the answer
  """
  if not use_llm_as_judge:
    return None,"API key is not available"

  judge_prompt=f"""
  You are an expert evaluator for question-answering AI systems.

INSTRUCTION (User Question):
{question}

REFERENCE ANSWER (Ground Truth):
{reference}

MODEL OUTPUT:
{prediction}

Evaluate the MODEL OUTPUT using the following criteria:

1. **Faithfulness**
   - Is the answer strictly supported by the reference?
   - Does it avoid hallucinations or unsupported claims?

2. **Accuracy**
   - Are the facts, numbers, conditions, and entities correct?
   - Are there any factual errors or misinterpretations?

3. **Completeness**
   - Does the answer cover all key points present in the reference?
   - Are any important details missing?

4. **Clarity & Coherence**
   - Is the answer well-structured and easy to understand?
   - Is the language concise and unambiguous?

5. **Safety & Appropriateness**
   - Does the answer avoid unsafe, misleading, or speculative content?
   - Does it avoid unnecessary medical/financial/legal advice beyond the reference?

---

### SCORING INSTRUCTIONS
- Assign a **single overall score from 1 to 5**, where:
  - **1** = Very poor (incorrect, hallucinated, unsafe)
  - **2** = Poor (major issues)
  - **3** = Fair (partially correct but incomplete or unclear)
  - **4** = Good (mostly correct with minor issues)
  - **5** = Excellent (fully correct, faithful, and clear)

---

### RESPONSE FORMAT (STRICT)
Respond using **exactly** the following format:

SCORE: [1–5]/5
REASONING: [2 concise sentences explaining the score]
  """

  response = judge_model.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are an expert evaluator for question-answering AI systems."},
            {"role": "user", "content": judge_prompt}
        ],
        temperature=0.0,
    )

  return response.choices[0].message.content.strip()


In [ ]:
i=0
for row in test_dataset.iterrows():
  answer,latency=run_baseline_model(row[1]["question"])

  print(f"Customer Question:{row[1]["question"]}")
  print(f"Assistant Answer:{answer}")
  print(f"Latency:{latency:.2f} ms")
  print(f"Reference Answer:{row[1]['anwser']}")
  print(f"LLM-as judge Score:{llm_as_judge(row[1]["question"],answer,row[1]['anwser'])}")
  print("="*60)

  i+=1
  if i==10:
    break

Customer Question:How many trading days must the price of the common stock exceed for at least 20 days during a 30-day period for a specific conversion condition?
Assistant Answer:The price of the common stock must exceed for at least 20 trading days during a 30-day period.
Latency:3044.66 ms
Reference Answer:The price of the common stock must exceed for at least 20 trading days during a 30-day period for a specific conversion condition.
LLM-as judge Score:SCORE: 5/5  
REASONING: The model output accurately reflects the key condition stated in the reference answer, maintaining both faithfulness and accuracy. It is clear, concise, and free from any unsupported claims or ambiguity.
Customer Question:Summarize the implications of failing to comply with healthcare-related regulations as outlined in the report.
Assistant Answer:Failing to comply with healthcare-related regulations could result in fines, penalties, or suspension of operations.
Latency:2760.86 ms
Reference Answer:Failing to c